In [1]:
import json
import re
with open('/Users/pranathi/Library/CloudStorage/OneDrive-ErasmusUniversityRotterdam/Masters/computer_science/TVs-all-merged.json', 'r') as file:
    data = json.load(file)
    


all_values = [item for sublist in data.values() for item in sublist]    
all_values_dict = {i: item for i, item in enumerate(all_values)}

def clean_all_values_dict_advanced(all_values_dict, remove_shops=True):
   
    def clean_text(text, remove_shops=True):
        if not isinstance(text, str):
            return text
        
        cleaned = text.lower()  # Lowercase first
        
        
        cleaned = re.sub(r'\s+', '', cleaned)  # Remove all spaces early
        
        
        inch_patterns = [
            (r'[\'"]{1,2}', 'inch'),           # " or ' or '' → inch
            (r'-inch', 'inch'),                 # -inch → inch (no \b needed after space removal)
            (r'inches', 'inch'),                # inches → inch
            (r'(\d+)in(?=\d|$)', r'\1inch'),    # 40in → 40inch (standalone "in")
        ]
        for pattern, replacement in inch_patterns:
            cleaned = re.sub(pattern, replacement, cleaned)
        
        # =====================================================================
        # STEP 3: Normalize HERTZ variations (simplified)
        # =====================================================================
        hz_patterns = [
            (r'hertz', 'hz'),                   # hertz → hz
            (r'-hz', 'hz'),                     # -hz → hz
        ]
        for pattern, replacement in hz_patterns:
            cleaned = re.sub(pattern, replacement, cleaned)
        
        # =====================================================================
        # STEP 4: Normalize RESOLUTION patterns (simplified)
        # =====================================================================
        resolution_patterns = [
            (r'4k', '2160p'),                   # 4k → 2160p
            (r'uhd', '2160p'),                  # uhd → 2160p
            (r'fullhd', '1080p'),               # fullhd → 1080p (spaces already removed)
            (r'hd(?=tv)', '720p'),              # hd before tv → 720p (but keep hdtv)
        ]
        for pattern, replacement in resolution_patterns:
            cleaned = re.sub(pattern, replacement, cleaned)
        
        # =====================================================================
        # STEP 5: Normalize LED/LCD/PLASMA/OLED (simplified)
        # =====================================================================
        display_patterns = [
            (r'led-lcd', 'led'),                # led-lcd → led
            (r'lcd/led', 'led'),                # lcd/led → led
            (r'led/lcd', 'led'),                # led/lcd → led
        ]
        for pattern, replacement in display_patterns:
            cleaned = re.sub(pattern, replacement, cleaned)
        
        # =====================================================================
        # STEP 6: Remove shop/website-specific noise (OPTIONAL)
        # =====================================================================
        # Only remove shops if remove_shops=True
        if remove_shops:
            website_noise = [
                # Main 4 shops - with domain extensions (most specific)
                r'newegg\.?com',
                r'amazon\.?com',
                r'bestbuy\.?com',
                r'best-buy\.?com',
                r'thenerds\.?net',
                # Other common shops
                
                # Shop names as standalone words (word boundaries)
                r'\bnewegg\b',
                r'\bamazon\b',
                r'\bbestbuy\b',
                r'\bbest\s*buy\b',
                r'\bthenerds\b',
                # Shop names at start/end (even without word boundaries after space removal)
                r'^newegg',
                r'^amazon',
                r'^bestbuy',
                r'^bestbuy',
                r'^thenerds',
                r'newegg$',
                r'amazon$',
                r'bestbuy$',  # Catches "bestbuy" at end (common pattern)
                r'bestbuy$',
                r'thenerds$',
                # Common prefixes/suffixes
                r'^refurbished:?',
                r'^certifiedrefurbished:?',
                r'-best\s*buy',
                r'-newegg',
                r'-amazon',
                r'best\s*buy$',
            ]
            for pattern in website_noise:
                cleaned = re.sub(pattern, '', cleaned, flags=re.IGNORECASE)
            
            # Additional cleanup: Remove any remaining shop fragments
            # This catches cases where shop name got partially removed or concatenated
            shop_fragments = ['newegg', 'amazon', 'bestbuy', 'thenerds']
            for fragment in shop_fragments:
                # Remove if it appears as a standalone word or at boundaries
                # This is especially important after space removal when shop names get concatenated
                cleaned = re.sub(rf'\b{fragment}\b', '', cleaned, flags=re.IGNORECASE)
                cleaned = re.sub(rf'^{fragment}', '', cleaned, flags=re.IGNORECASE)
                cleaned = re.sub(rf'{fragment}$', '', cleaned, flags=re.IGNORECASE)
        
        # =====================================================================
        # STEP 7: Remove noise characters (punctuation, special chars)
        # =====================================================================
        cleaned = re.sub(r'[^\w]', '', cleaned)  # Remove all non-alphanumeric
        
        return cleaned
    
    def clean_dict(d, remove_shops=True):
        cleaned = {}
        for k, v in d.items():
            # SKIP modelID completely - preserve original key and value
            if k.lower() == 'modelid':
                cleaned[k] = v
                continue
            
            key_clean = clean_text(k, remove_shops=remove_shops)
            if isinstance(v, str):
                cleaned[key_clean] = clean_text(v, remove_shops=remove_shops)
            elif isinstance(v, dict):
                cleaned[key_clean] = clean_dict(v, remove_shops=remove_shops)
            else:
                cleaned[key_clean] = v
        return cleaned
    
    return {idx: clean_dict(item, remove_shops=remove_shops) for idx, item in all_values_dict.items()}



In [25]:
# ============================================================================
# COUNT TOTAL DUPLICATE PAIRS IN ORIGINAL DATA (Using modelID)
# ============================================================================
from collections import defaultdict

# Build ground truth clusters from modelID
modelID_to_indices = defaultdict(list)

for idx, item in all_values_dict.items():
    if 'modelID' in item and item['modelID']:
        modelID_to_indices[item['modelID']].append(idx)

# Keep only modelIDs with 2+ products (actual duplicates)
duplicate_clusters = {mid: set(idxs) for mid, idxs in modelID_to_indices.items() if len(idxs) > 1}

# Calculate total duplicate pairs
# For each cluster with n products, there are n*(n-1)/2 pairs
total_duplicate_pairs = 0
cluster_sizes = []

for modelID, indices in duplicate_clusters.items():
    n = len(indices)
    pairs_in_cluster = n * (n - 1) // 2
    total_duplicate_pairs += pairs_in_cluster
    cluster_sizes.append(n)

# Display results
print("="*70)
print("DUPLICATE PAIRS ANALYSIS (Based on modelID)")
print("="*70)
print(f"\nTotal products in dataset: {len(all_values_dict)}")
print(f"Unique modelIDs: {len(modelID_to_indices)}")
print(f"ModelIDs with duplicates (2+ products): {len(duplicate_clusters)}")
print(f"ModelIDs with single product (no duplicates): {len(modelID_to_indices) - len(duplicate_clusters)}")
print(f"\nTotal duplicate pairs: {total_duplicate_pairs}")
print(f"\nCluster size distribution:")
print(f"  Mean cluster size: {sum(cluster_sizes) / len(cluster_sizes) if cluster_sizes else 0:.2f}")
print(f"  Min cluster size: {min(cluster_sizes) if cluster_sizes else 0}")
print(f"  Max cluster size: {max(cluster_sizes) if cluster_sizes else 0}")

# Show distribution of cluster sizes
from collections import Counter
size_distribution = Counter(cluster_sizes)
print(f"\nCluster size frequency:")
for size in sorted(size_distribution.keys()):
    count = size_distribution[size]
    pairs = size * (size - 1) // 2
    print(f"  Size {size}: {count} clusters ({pairs * count} pairs)")

# Calculate what fraction of all possible pairs are duplicates
total_possible_pairs = len(all_values_dict) * (len(all_values_dict) - 1) // 2
fraction_duplicates = total_duplicate_pairs / total_possible_pairs if total_possible_pairs > 0 else 0

print(f"\nFraction of all possible pairs that are duplicates:")
print(f"  {fraction_duplicates:.6f} ({fraction_duplicates*100:.4f}%)")
print(f"  (Total possible pairs: {total_possible_pairs:,})")
print(f"  (Duplicate pairs: {total_duplicate_pairs:,})")

print("="*70)

# Store for later use
ground_truth_duplicate_pairs = total_duplicate_pairs
ground_truth_duplicate_clusters = duplicate_clusters


DUPLICATE PAIRS ANALYSIS (Based on modelID)

Total products in dataset: 1624
Unique modelIDs: 1262
ModelIDs with duplicates (2+ products): 329
ModelIDs with single product (no duplicates): 933

Total duplicate pairs: 399

Cluster size distribution:
  Mean cluster size: 2.10
  Min cluster size: 2
  Max cluster size: 4

Cluster size frequency:
  Size 2: 300 clusters (300 pairs)
  Size 3: 25 clusters (75 pairs)
  Size 4: 4 clusters (24 pairs)

Fraction of all possible pairs that are duplicates:
  0.000303 (0.0303%)
  (Total possible pairs: 1,317,876)
  (Duplicate pairs: 399)


In [2]:
import numpy as np
from sklearn.utils import resample

def bootstrap_train_test_split(n_samples):
    
    # Draw bootstrap sample with replacement using sklearn
    train_indices = resample(range(n_samples), n_samples=n_samples, 
                            replace=True)
    train_indices = np.array(train_indices)
    
    # Out-of-sample instances (not selected in bootstrap)
    all_indices = set(range(n_samples))
    train_set = set(train_indices)
    test_indices = np.array(sorted(all_indices - train_set))
    
    return train_indices, test_indices

def run_bootstrap_iterations(n_samples, n_bootstraps=5):
    
    bootstrap_splits = []
    
    for i in range(n_bootstraps):
        train_idx, test_idx = bootstrap_train_test_split(n_samples)
        bootstrap_splits.append((train_idx, test_idx))
        
        if (i + 1) % 10 == 0 or i == 0:
            train_pct = len(np.unique(train_idx)) / n_samples * 100
            test_pct = len(test_idx) / n_samples * 100
            print(f"Bootstrap {i+1}/{n_bootstraps}: "
                  f"{len(np.unique(train_idx))} unique train ({train_pct:.1f}%), "
                  f"{len(test_idx)} test ({test_pct:.1f}%)")
    
    return bootstrap_splits

def apply_bootstrap_to_data(data, train_indices, test_indices):
    
    if isinstance(data, np.ndarray):
        train_data = data[train_indices]
        test_data = data[test_indices]
    elif isinstance(data, list):
        train_data = [data[i] for i in train_indices]
        test_data = [data[i] for i in test_indices]
    else:  # Assume pandas DataFrame/Series
        train_data = data.iloc[train_indices]
        test_data = data.iloc[test_indices]
    
    return train_data, test_data


In [3]:

import re

def split_and_extract_model_words(title):
    title = title.lower()

    # 1) Find base model tokens containing both letters and digits
    base_pattern = r"[a-z0-9]*(([0-9]+[^0-9,\s]+)|([^0-9,\s]+[0-9]+))[a-z0-9]*"
    base_tokens = [m.group(0) for m in re.finditer(base_pattern, title)]

    refined_tokens = []
    for tok in base_tokens:
        # 2) Split token further into pieces at digit/letter boundaries
        #    e.g. "32inch720p" -> ["32inch", "720p"]
        parts = re.findall(r"[0-9]+[a-z]+|[a-z]+[0-9]+|[0-9]+|[a-z]+", tok)
        # keep only parts that contain both letters and digits (model-like)
        for p in parts:
            if re.search(r"[a-z]", p) and re.search(r"[0-9]", p):
                refined_tokens.append(p)

    return {"model_words": refined_tokens}




def extract_model_words_from_title(title):
   
    if not isinstance(title, str):
        return set()
    
    title = title.lower()
    
    # Paper's pattern for title model words
    base_pattern = r"[a-z0-9]*(([0-9]+[^0-9,\s]+)|([^0-9,\s]+[0-9]+))[a-z0-9]*"
    base_tokens = [m.group(0) for m in re.finditer(base_pattern, title)]
    
    refined_tokens = []
    for tok in base_tokens:
        # Split token further at digit/letter boundaries
        parts = re.findall(r"[0-9]+[a-z]+|[a-z]+[0-9]+|[0-9]+|[a-z]+", tok)
        for p in parts:
            if re.search(r"[a-z]", p) and re.search(r"[0-9]", p):
                refined_tokens.append(p)
    
    return set(refined_tokens)


def extract_model_words_from_value(value):
    
    if not isinstance(value, str):
        return set()
    
    value = value.lower().strip()
    model_words = set()
    
    # Split by common separators to handle composite values like "36.8inchx24.4inch"
    parts = re.split(r'[x×\(\)]', value)
    
    for part in parts:
        part = part.strip()
        if not part:
            continue
            
        # Pattern: decimal number with optional alphabetic suffix
        # Match: "19.2lb", "40inch", "120hz", "1080p", "19.5", "120"
        match = re.match(r'^(\d+(?:\.\d+)?)[a-z]*$', part)
        if match:
            # Extract only the numeric part (remove non-numeric suffix)
            numeric_part = match.group(1)
            model_words.add(numeric_part)
    
    return model_words


def extract_all_model_words_paper_method(item):
    """
    Extract model words from brands, title, and the number 1080.
    
    1. MW_title: Extract from title only
    2. MW_brand: Extract brand name as a model word
    3. MW_1080: Extract "1080" if it appears in the title
    
    Returns union of all model words.
    """
    mw_title = set()
    mw_brand = set()
    mw_1080 = set()
    
    # Step 1: Extract MW_title from title
    title = item.get('title', '') or ''
    mw_title.update(extract_model_words_from_title(title))
    
    # Step 2: Extract brand as a model word
    # Brand is critical for matching - products with same brand are more likely duplicates
    brand_list = [
        "samsung", "lg", "sony", "philips", "panasonic", "sharp", "toshiba",
        "vizio", "hisense", "tcl", "insignia", "sanyo", "magnavox", "westinghouse",
        "element", "rca", "haier", "funai", "sceptre", "proscan", "coby",
        "supersonic", "naxa", "dynex", "emerson", "sylvania", "jvc", "hitachi"
    ]
    
    # Check title for brand
    title_lower = title.lower()
    for brand in brand_list:
        if brand in title_lower:
            mw_brand.add(brand)
            break  # Only add first matching brand
    
    # Step 3: Extract "1080" from title if present
    # Look for "1080" as a standalone number or as part of "1080p"
    if re.search(r'1080', title_lower):
        mw_1080.add('1080')
    
    # Return union of all sets (title, brand, and 1080 model words)
    # Filter out pure numeric strings (except 1080 which we want to keep)
    all_mw = mw_title | mw_brand | mw_1080
    # Keep 1080 but filter out other pure numeric strings
    filtered_mw = [mw for mw in all_mw if mw == '1080' or not re.match(r'^\d+(\.\d+)?$', mw)]
    return filtered_mw


def extract_model_words_title_only(item):
    """
    Extract model words ONLY from title (no brand, no 1080).
    This is for comparison with the full method.
    
    Returns only model words extracted from the title.
    """
    mw_title = set()
    
    # Extract MW_title from title only
    title = item.get('title', '') or ''
    mw_title.update(extract_model_words_from_title(title))
    
    # Return only title model words (no brand, no 1080)
    # Filter out pure numeric strings
    filtered_mw = [mw for mw in mw_title if not re.match(r'^\d+(\.\d+)?$', mw)]
    return filtered_mw




In [4]:

def create_binary_vectors(titles_model_words, vocab, word_to_index):
    binary_vectors = []
    for words in titles_model_words:
        vector = [0] * len(vocab)
        for word in words:
            if word in word_to_index:
                vector[word_to_index[word]] = 1
        binary_vectors.append(vector)
    
    P = np.array(binary_vectors)
    print(f"Binary vectors shape: {P.shape}")
    return P


In [5]:
def next_prime(n):
    if n <= 2:
        return 2
    n = n if n % 2 != 0 else n + 1  # Make odd
    while True:
        is_prime = True
        for i in range(3, int(n**0.5) + 1, 2):
            if n % i == 0:
                is_prime = False
                break
        if is_prime:
            return n
        n += 2

In [6]:
def compute_minhash_signatures(P, n_hash_functions=None):
    num_products, vocab_size = P.shape
    
    if n_hash_functions is None:
        n = int(0.4* vocab_size)  # Increased from 0.4 to 0.5
        n = max(n, 2)
    
    # Choose large prime p > vocab_size
    p = next_prime(vocab_size * vocab_size + 1)  # Ensure p > vocab_size^2
    m = vocab_size
    
    print(f"Using p={p}, hash_functions={n}, vocab_size={vocab_size}")
    
    # Generate a,b pairs for n hash functions
    a_params = np.random.randint(1, p, size=n)
    b_params = np.random.randint(0, p, size=n)
    
    # OPTIMIZATION: Use full prime range instead of vocab_size to reduce collisions
    max_hash_value = p - 1
    S = np.full((n, num_products), max_hash_value, dtype=np.int64)
    
    non_zero_positions = [np.where(P[v] == 1)[0] for v in range(num_products)]
    
    for i in range(n):
        a, b = a_params[i], b_params[i]
        for v in range(num_products):
            product_set = non_zero_positions[v]
            
            if len(product_set) > 0:
                # Use only % p which gives range [0, p-1] instead of [0, vocab_size-1]
                hashes = ((a * product_set + b) % p)
                min_hash = np.min(hashes)
                S[i, v] = min_hash
            
    S_normalized = (S % vocab_size).astype(int)
    
    print(f"Signature matrix shape: {S_normalized.shape}")
    return S_normalized, (a_params, b_params, p, vocab_size)


In [7]:
def count_true_duplicates_in_candidates(dict_data, candidate_pairs, return_set=False):
    from collections import defaultdict
    
    # Build ground truth clusters from modelID
    modelID_to_indices = defaultdict(list)
    for idx, item in dict_data.items():
        if 'modelID' in item:
            modelID_to_indices[item['modelID']].append(idx)
    
    # Keep only modelIDs with 2+ products (actual duplicates)
    duplicate_clusters = {mid: set(idxs) for mid, idxs in modelID_to_indices.items() if len(idxs) > 1}
    
    # Find true duplicates within candidate_pairs
    true_duplicates_in_candidates = set()
    for i, j in candidate_pairs:
        a, b = sorted((i, j))
        for cluster in duplicate_clusters.values():
            if a in cluster and b in cluster:
                true_duplicates_in_candidates.add((a, b))
                break
    
    n_true_duplicates = len(true_duplicates_in_candidates)
    
    if return_set:
        return n_true_duplicates, true_duplicates_in_candidates
    else:
        return n_true_duplicates


# Example usage:
# n_true = count_true_duplicates_in_candidates(dict_test, candidate_pairs)
# n_true, true_set = count_true_duplicates_in_candidates(dict_test, candidate_pairs, return_set=True)


In [8]:
# ============================================================================
# FUNCTION: Find Best Threshold Using F1 Score on Training Set
# ============================================================================
def find_best_threshold_on_training_set(dict_train, model_words_train, vocab, word_to_index, 
                                        thresholds=[0.4, 0.5], f1_range=(0.0, 0.2), verbose=True):
    
    from collections import defaultdict
    
    if verbose:
        print("\n" + "="*70)
        print("FINDING BEST THRESHOLD ON TRAINING SET")
        print("="*70)
        f1_max = f1_range[1] if isinstance(f1_range, tuple) else f1_range
        print(f"F1 score upper bound constraint: F1 <= {f1_max:.2f}")
        print(f"Testing thresholds: {thresholds}")
        print(f"Training set size: {len(dict_train)} products")
    
    # Build ground truth for training set
    modelID_to_indices_train = defaultdict(list)
    for idx, item in dict_train.items():
        if 'modelID' in item:
            modelID_to_indices_train[item['modelID']].append(idx)
    
    # Keep only modelIDs with 2+ products (actual duplicates)
    duplicate_clusters_train = {mid: set(idxs) for mid, idxs in modelID_to_indices_train.items() if len(idxs) > 1}
    
    # Calculate total true duplicate pairs in training set
    total_true_pairs_train = sum(len(idxs) * (len(idxs) - 1) // 2 for idxs in duplicate_clusters_train.values())
    
    if verbose:
        print(f"Total true duplicate pairs in training set: {total_true_pairs_train}")
        print(f"Duplicate clusters: {len(duplicate_clusters_train)}")
        print("\nTesting thresholds...")
        print(f"{'Threshold':<12} {'Candidates':<12} {'TP':<6} {'PQ':<10} {'PC':<10} {'F1':<10}")
        print("-" * 70)
    
    # Test each threshold
    threshold_results = {}
    best_threshold = None
    best_f1 = -1
    
    # Create binary vectors and MinHash signatures once (same for all thresholds)
    A_train = create_binary_vectors(model_words_train, vocab, word_to_index)
    L_train, _ = compute_minhash_signatures(A_train)
    product_ids_train = list(dict_train.keys())
    n_hash_functions = L_train.shape[0]
    
    if verbose:
        print(f"Signature matrix shape: {L_train.shape}")
        print(f"Number of hash functions: {n_hash_functions}")
    
    for t in thresholds:
        if verbose:
            print(f"\nTesting threshold {t}...")
        
        # Calculate (b, r) for this threshold using theoretical formula: t = (1/b)^(1/r)
        # Subject to constraint: b * r == n_hash_functions
        best_b, best_r = optimize_lsh_parameters(n_hash_functions, t, min_prob=0.5)
        
        if verbose:
            t_achieved = (1 / best_b) ** (1 / best_r)
            print(f"    Theoretical (b, r) = ({best_b}, {best_r}) for threshold {t}")
            print(f"    Achieved threshold: {t_achieved:.6f} (error: {abs(t - t_achieved):.6f})")
        
        # Run LSH with theoretical (b, r) for this threshold
        candidate_pairs = lsh_banding(L_train, product_ids_train, t=t, b=best_b, r=best_r, verbose=False)
        
        # Count true duplicates using the function
        TP = count_true_duplicates_in_candidates(dict_train, candidate_pairs)
        
        # Calculate metrics
        PQ = TP / len(candidate_pairs) if len(candidate_pairs) > 0 else 0.0
        PC = TP / total_true_pairs_train if total_true_pairs_train > 0 else 0.0
        F1 = 2 * (PQ * PC) / (PQ + PC) if (PQ + PC) > 0 else 0.0
        
        threshold_results[t] = {
            'candidate_pairs': len(candidate_pairs),
            'TP': TP,
            'PQ': PQ,
            'PC': PC,
            'F1': F1,
            'best_b': best_b,
            'best_r': best_r
        }
        
        if verbose:
            f1_max = f1_range[1] if isinstance(f1_range, tuple) else f1_range
            f1_status = "✓" if F1 <= f1_max else "✗"
            print(f"{t:<12.2f} {len(candidate_pairs):<12} {TP:<6} {PQ:<10.6f} {PC:<10.6f} {F1:<10.6f} (b={best_b}, r={best_r}) {f1_status}")
        
        # Update best threshold (only if F1 <= f1_max)
        f1_max = f1_range[1] if isinstance(f1_range, tuple) else f1_range
        if F1 <= f1_max and F1 > best_f1:
            best_f1 = F1
            best_threshold = t
    
    if verbose:
        print("-" * 70)
        if best_threshold is not None:
            f1_max = f1_range[1] if isinstance(f1_range, tuple) else f1_range
            if best_f1 <= f1_max:
                print(f"\nBest threshold: {best_threshold} (F1 = {best_f1:.6f}) [F1 <= {f1_max:.2f} ✓]")
            else:
                print(f"\nBest threshold: {best_threshold} (F1 = {best_f1:.6f}) [F1 > {f1_max:.2f} ✗ - no threshold found with F1 <= {f1_max:.2f}]")
        else:
            print(f"\nNo valid threshold found!")
        print("="*70)
    
    return best_threshold, best_f1, threshold_results

def find_best_lsh_bands_rows_for_threshold(L_train, product_ids_train, dict_train, threshold, 
                                          n_hash_functions, max_configs=30, f1_range=(0.0, 0.2), verbose=True):
   
    from collections import defaultdict
    import numpy as np
    
    # Build ground truth for training set
    modelID_to_indices_train = defaultdict(list)
    for idx, item in dict_train.items():
        if 'modelID' in item:
            modelID_to_indices_train[item['modelID']].append(idx)
    
    duplicate_clusters_train = {mid: set(idxs) for mid, idxs in modelID_to_indices_train.items() if len(idxs) > 1}
    total_true_pairs_train = sum(len(idxs) * (len(idxs) - 1) // 2 for idxs in duplicate_clusters_train.values())
    
    # Generate candidate (b, r) pairs
    # Constraint: b * r == n_hash_functions (STRICT EQUALITY)
    configs = []
    min_r = 2
    max_r = n_hash_functions##min(int(np.sqrt(n_hash_functions)) + 10, n_hash_functions // 2)
    
    for r in range(min_r, max_r + 1):
        if n_hash_functions % r == 0:  # Only if r divides n_hash_functions exactly
            b = n_hash_functions // r
            if b >= 1 and b * r == n_hash_functions:  # STRICT equality: b * r == n
                configs.append((b, r))
    
    # Limit number of configurations to test
    if len(configs) > max_configs:
        # Sample evenly across the range
        step = len(configs) // max_configs
        configs = configs[::max(1, step)][:max_configs]
    
    if verbose:
        print(f"    Testing {len(configs)} (b, r) configurations for threshold {threshold}...")
    
    best_b, best_r, best_f1 = None, None, -1
    config_results = {}
    
    for b, r in configs:
        # Run LSH with this configuration
        candidate_pairs = lsh_banding(L_train, product_ids_train, t=threshold, b=b, r=r, verbose=False)
        
        # Evaluate
        TP = count_true_duplicates_in_candidates(dict_train, candidate_pairs)
        PQ = TP / len(candidate_pairs) if len(candidate_pairs) > 0 else 0.0
        PC = TP / total_true_pairs_train if total_true_pairs_train > 0 else 0.0
        F1 = 2 * (PQ * PC) / (PQ + PC) if (PQ + PC) > 0 else 0.0
        
        config_results[(b, r)] = {
            'candidate_pairs': len(candidate_pairs),
            'TP': TP,
            'PQ': PQ,
            'PC': PC,
            'F1': F1
        }
        
        # Only consider configurations with F1 <= f1_max (upper bound constraint)
        # Select the best F1 that satisfies F1 <= f1_max
        f1_max = f1_range[1] if isinstance(f1_range, tuple) else f1_range
        if F1 <= f1_max and F1 > best_f1:
            best_f1 = F1
            best_b = b
            best_r = r
    
    if verbose:
        if best_b is not None:
            f1_max = f1_range[1] if isinstance(f1_range, tuple) else f1_range
            if best_f1 <= f1_max:
                print(f"      Best (b, r) = ({best_b}, {best_r}) with F1 = {best_f1:.6f} [F1 <= {f1_max:.2f} ✓]")
            else:
                print(f"      Best (b, r) = ({best_b}, {best_r}) with F1 = {best_f1:.6f} [F1 > {f1_max:.2f} ✗]")
        else:
            f1_max = f1_range[1] if isinstance(f1_range, tuple) else f1_range
            print(f"      No valid (b, r) configuration found with F1 <= {f1_max:.2f}!")
    
    return best_b, best_r, best_f1, config_results



In [9]:
####new
import numpy as np
import hashlib
from collections import defaultdict

def lsh_banding(M, product_ids, t=0.5, b=None, r=None, verbose=True):
    n_rows, num_products = M.shape
    
    # Optimize b and r if not provided
    if b is None or r is None:
        b, r = optimize_lsh_parameters(n_rows, t)
    
    # Validate constraint: b * r == n_rows (STRICT)
    if b * r != n_rows:
        raise ValueError(f"LSH constraint violation: b * r ({b} * {r} = {b*r}) must equal n_rows ({n_rows})")
    
    if verbose:
        prob_at_t = 1 - (1 - t**r)**b
    
    candidate_pairs = set()
    
    # Process each band
    # Since b * r == n_rows, end_row will always be start_row + r (no need for min)
    for band_idx in range(b):
        start_row = band_idx * r
        end_row = start_row + r  # Since b * r == n_rows, this is always <= n_rows
        
        # Extract band signatures
        band_matrix = M[start_row:end_row, :]
        
        # Hash products into buckets
        buckets = defaultdict(list)
        
        for prod_idx in range(num_products):
            # Get signature for this product in this band
            signature = tuple(band_matrix[:, prod_idx].tolist())
            
            # Hash the signature (use band index for independence)
            h = hashlib.md5()
            h.update(str(band_idx).encode())
            h.update(str(signature).encode())
            bucket_hash = h.hexdigest()
            
            buckets[bucket_hash].append(prod_idx)
        
        # Generate candidate pairs from buckets with 2+ products
        for bucket_products in buckets.values():
            if len(bucket_products) >= 2:
                for i in range(len(bucket_products)):
                    for j in range(i + 1, len(bucket_products)):
                        idx_i = bucket_products[i]
                        idx_j = bucket_products[j]
                        pair = tuple(sorted([product_ids[idx_i], product_ids[idx_j]]))
                        candidate_pairs.add(pair)
    
    if verbose:
        print(f"\nResults:")
        print(f"  Candidate pairs: {len(candidate_pairs)}")
        max_possible = num_products * (num_products - 1) // 2
        print(f"  vs. All possible pairs: {max_possible}")
        print(f"  Reduction: {100 * (1 - len(candidate_pairs) / max_possible):.1f}%")
    
    return candidate_pairs


def optimize_lsh_parameters(n, t, min_prob=0.5):
    """
    Optimize b (bands) and r (rows per band) for given threshold t.
    
    Constraints:
    - ENFORCES: b * r == n (strict equality)
    - APPROXIMATES: t = (1/b)^(1/r) (minimizes error from desired threshold)
    """
    import numpy as np
    
    best_b, best_r = None, None
    best_error = float('inf')
    
    # Try all possible r values
    for r in range(1, min(n + 1, 100)):  # Increased range for better optimization
        # Calculate b from the LSH equation: t = (1/b)^(1/r)
        # Rearranging: b = 1/(t^r)
        b_from_equation = 1 / (t ** r)
        
        # ENFORCE: b * r == n (STRICT EQUALITY)
        # Only consider r values that divide n exactly
        if n % r != 0:
            continue
        
        b = n // r  # This ensures b * r == n exactly
        if b < 1:
            continue
        
        # APPROXIMATE: t = (1/b)^(1/r) (minimize error from desired threshold)
        # Calculate the threshold that would be achieved with this (b, r)
        t_achieved = (1 / b) ** (1 / r)
        
        # Calculate error: how far is t_achieved from desired t?
        error = abs(t - t_achieved)
        
        b_final = b
        t_final = t_achieved
        error_final = error
        
        # Calculate probability at threshold for validation
        prob_at_t = 1 - (1 - t_final**r)**b_final
        
        # Only consider configurations that meet minimum probability requirement
        if prob_at_t < min_prob:
            continue
        
        # Select configuration with smallest error from desired threshold
        if error_final < best_error:
            best_error = error_final
            best_b = b_final
            best_r = r
    
    # Fallback if no valid configuration found
    # Still enforce b * r == n strictly
    if best_b is None:
        # Find r that divides n and gives best approximation to threshold
        best_error_fallback = float('inf')
        for r in range(1, min(n + 1, 100)):
            if n % r != 0:  # Must divide n exactly
                continue
            b = n // r
            if b < 1:
                continue
            t_calc = (1 / b) ** (1 / r)
            error_calc = abs(t - t_calc)
            if error_calc < best_error_fallback:
                best_error_fallback = error_calc
                best_r = r
                best_b = b
    
    # Final fallback: if still no valid configuration, use r=1, b=n (guaranteed to work)
    if best_b is None:
        best_r = 1
        best_b = n
        if verbose:
            print(f"WARNING: No optimal (b, r) found, using fallback: b={best_b}, r={best_r}")
    
    return best_b, best_r



In [10]:
# ============================================================================
# AGGLOMERATIVE CLUSTERING FUNCTIONS
# ============================================================================
from sklearn.cluster import AgglomerativeClustering
import numpy as np
from collections import defaultdict

def compute_tmwm_similarity(item1, item2):
    """
    Compute TMWM (Title Model Words Method) similarity between two products.
    Uses overlap coefficient on model word sets: |A ∩ B| / min(|A|, |B|)
    
    Parameters:
    -----------
    item1 : dict
        First product item (must have 'title' key)
    item2 : dict
        Second product item (must have 'title' key)
    
    Returns:
    --------
    similarity : float
        TMWM similarity between 0.0 and 1.0
    """
    # Extract model words from titles
    title1 = item1.get('title', '') or ''
    title2 = item2.get('title', '') or ''
    
    # Extract model words using the existing function
    mw1 = extract_model_words_from_title(title1)
    mw2 = extract_model_words_from_title(title2)
    
    # If either has no model words, return 0 similarity
    if not mw1 or not mw2:
        return 0.0
    
    # Compute overlap coefficient: intersection / min(size)
    intersection = len(mw1 & mw2)
    min_size = min(len(mw1), len(mw2))
    
    if min_size == 0:
        return 0.0
    
    similarity = intersection / min_size
    return min(1.0, similarity)  # Ensure it's between 0 and 1


def build_similarity_matrix_from_pairs(filtered_pairs, all_product_ids, dict_data):
    """
    Build a similarity matrix from filtered pairs using TMWM similarity.
    Products in filtered_pairs get their actual TMWM similarity score,
    others get 0.
    
    Parameters:
    -----------
    filtered_pairs : set of tuples
        Set of (product_id1, product_id2) pairs
    all_product_ids : list
        List of all product IDs to include in the matrix
    dict_data : dict
        Dictionary mapping product IDs to product data (must have 'title' key)
    
    Returns:
    --------
    similarity_matrix : np.ndarray
        TMWM similarity matrix (values between 0.0 and 1.0)
    product_id_to_index : dict
        Mapping from product ID to matrix index
    """
    n = len(all_product_ids)
    product_id_to_index = {pid: idx for idx, pid in enumerate(all_product_ids)}
    
    # Initialize similarity matrix (all zeros)
    similarity_matrix = np.zeros((n, n), dtype=float)
    
    # Set diagonal to 1.0 (self-similarity)
    np.fill_diagonal(similarity_matrix, 1.0)
    
    # Compute TMWM similarity for each pair
    computed = 0
    for i, j in filtered_pairs:
        if i in product_id_to_index and j in product_id_to_index:
            idx_i = product_id_to_index[i]
            idx_j = product_id_to_index[j]
            
            # Get product data
            item_i = dict_data.get(i)
            item_j = dict_data.get(j)
            
            if item_i and item_j:
                # Compute TMWM similarity
                tmwm_sim = compute_tmwm_similarity(item_i, item_j)
                similarity_matrix[idx_i, idx_j] = tmwm_sim
                similarity_matrix[idx_j, idx_i] = tmwm_sim  # Symmetric
                computed += 1
    
    print(f"  Computed TMWM similarity for {computed} pairs")
    
    return similarity_matrix, product_id_to_index


def apply_agglomerative_clustering(filtered_pairs, all_product_ids, dict_data,
                                   linkage='complete', n_clusters=None, 
                                   distance_threshold=None):
    """
    Apply Agglomerative Clustering to filtered pairs using TMWM similarity.
    
    Parameters:
    -----------
    filtered_pairs : set of tuples
        Set of (product_id1, product_id2) pairs
    all_product_ids : list
        List of all product IDs
    dict_data : dict
        Dictionary mapping product IDs to product data (must have 'title' key)
    linkage : str
        Linkage criterion ('complete', 'single', 'average', 'ward')
        'complete' = maximum linkage (double linkage)
    n_clusters : int, optional
        Number of clusters to form
    distance_threshold : float, optional
        Distance threshold for cutting the tree
    
    Returns:
    --------
    cluster_labels : np.ndarray
        Cluster labels for each product
    clustering_model : AgglomerativeClustering
        The fitted clustering model
    """
    if len(filtered_pairs) == 0:
        # If no pairs, each product is its own cluster
        return np.zeros(len(all_product_ids)), None
    
    # Build similarity matrix using TMWM
    similarity_matrix, product_id_to_index = build_similarity_matrix_from_pairs(
        filtered_pairs, all_product_ids, dict_data
    )
    
    # Convert similarity to distance (for clustering, we need distance)
    # Similarity 1 -> distance 0, similarity 0 -> distance 1
    distance_matrix = 1.0 - similarity_matrix
    
    # Apply clustering
    # If n_clusters is None, use distance_threshold
    clustering = AgglomerativeClustering(
        n_clusters=n_clusters,
        metric='precomputed',
        linkage=linkage,
        distance_threshold=distance_threshold,
        compute_full_tree=True
    )
    
    cluster_labels = clustering.fit_predict(distance_matrix)
    
    return cluster_labels, clustering


def evaluate_clustering_f1_score(cluster_labels, dict_data, product_ids):
    """
    Evaluate clustering results using F1 score based on ground truth modelID.
    
    Parameters:
    -----------
    cluster_labels : np.ndarray
        Cluster labels from clustering
    dict_data : dict
        Dictionary mapping product IDs to product data
    product_ids : list
        List of product IDs in the same order as cluster_labels
    
    Returns:
    --------
    f1_score : float
        F1 score for the clustering
    precision : float
        Precision (Pair Quality)
    recall : float
        Recall (Pair Completeness)
    """
    # Build ground truth clusters from modelID
    modelID_to_indices = defaultdict(list)
    for idx, product_id in enumerate(product_ids):
        item = dict_data.get(product_id)
        if item and 'modelID' in item:
            modelID_to_indices[item['modelID']].append(idx)
    
    # Keep only modelIDs with 2+ products (actual duplicates)
    true_clusters = {mid: set(idxs) for mid, idxs in modelID_to_indices.items() if len(idxs) > 1}
    
    # Build predicted clusters from cluster labels
    predicted_clusters = defaultdict(set)
    for idx, label in enumerate(cluster_labels):
        predicted_clusters[label].add(idx)
    
    # Remove singleton clusters (they don't contribute to pairs)
    predicted_clusters = {label: cluster for label, cluster in predicted_clusters.items() if len(cluster) > 1}
    
    # Calculate true positive pairs (pairs that are in both true and predicted clusters)
    true_positive_pairs = set()
    
    # For each predicted cluster
    for pred_cluster in predicted_clusters.values():
        # Check if all products in this cluster have the same modelID
        model_ids_in_cluster = set()
        for idx in pred_cluster:
            product_id = product_ids[idx]
            item = dict_data.get(product_id)
            if item and 'modelID' in item:
                model_ids_in_cluster.add(item['modelID'])
        
        # If all products have the same modelID, all pairs are true positives
        if len(model_ids_in_cluster) == 1:
            model_id = list(model_ids_in_cluster)[0]
            if model_id in true_clusters:
                # All pairs within this cluster are true positives
                cluster_list = sorted(list(pred_cluster))
                for i in range(len(cluster_list)):
                    for j in range(i + 1, len(cluster_list)):
                        true_positive_pairs.add((cluster_list[i], cluster_list[j]))
    
    # Calculate total true pairs
    total_true_pairs = sum(len(idxs) * (len(idxs) - 1) // 2 for idxs in true_clusters.values())
    
    # Calculate total predicted pairs
    total_predicted_pairs = sum(len(cluster) * (len(cluster) - 1) // 2 for cluster in predicted_clusters.values())
    
    # Calculate metrics
    TP = len(true_positive_pairs)
    precision = TP / total_predicted_pairs if total_predicted_pairs > 0 else 0.0
    recall = TP / total_true_pairs if total_true_pairs > 0 else 0.0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return f1_score, precision, recall, TP, total_predicted_pairs, total_true_pairs


def find_best_clustering_distance_threshold(filtered_pairs_train, all_product_ids_train, dict_train,
                                            linkage='complete',
                                            distance_thresholds=[0.2, 0.3, 0.4, 0.5, 0.6, 0.7],
                                            f1_range=(0.1, 0.2),
                                            verbose=True):
    """
    Find the best distance_threshold for agglomerative clustering on training set.
    Includes safeguards against overfitting by preferring thresholds within F1 range.
    
    Parameters:
    -----------
    filtered_pairs_train : set of tuples
        Training set filtered pairs
    all_product_ids_train : list
        List of all training product IDs
    dict_train : dict
        Training data dictionary
    linkage : str
        Linkage criterion (default: 'complete')
    distance_thresholds : list
        List of distance thresholds to test
    f1_range : tuple
        Desired F1 range (min, max) - only consider thresholds in this range
    verbose : bool
        Whether to print progress
    
    Returns:
    --------
    best_threshold : float
        Best distance threshold
    best_f1_train : float
        Best F1 score on training set
    threshold_results : dict
        Results for all tested thresholds
    """
    from collections import defaultdict
    
    if verbose:
        print("\n" + "="*70)
        print("FINDING BEST CLUSTERING DISTANCE THRESHOLD ON TRAINING SET")
        print("="*70)
        f1_max = f1_range[1] if isinstance(f1_range, tuple) else f1_range
        print(f"F1 score upper bound constraint: F1 <= {f1_max:.2f}")
        print(f"Testing distance thresholds: {distance_thresholds}")
        print(f"Linkage: {linkage}")
        print(f"Training set size: {len(all_product_ids_train)} products")
        print(f"Filtered pairs: {len(filtered_pairs_train)}")
    
    # Build ground truth for training set
    modelID_to_indices_train = defaultdict(list)
    for idx, product_id in enumerate(all_product_ids_train):
        item = dict_train.get(product_id)
        if item and 'modelID' in item:
            modelID_to_indices_train[item['modelID']].append(idx)
    
    duplicate_clusters_train = {mid: set(idxs) for mid, idxs in modelID_to_indices_train.items() if len(idxs) > 1}
    total_true_pairs_train = sum(len(idxs) * (len(idxs) - 1) // 2 for idxs in duplicate_clusters_train.values())
    
    if verbose:
        print(f"Total true duplicate pairs in training set: {total_true_pairs_train}")
        print(f"Duplicate clusters: {len(duplicate_clusters_train)}")
        print("\nTesting thresholds...")
        print(f"{'Threshold':<12} {'Clusters':<10} {'TP':<6} {'PQ':<10} {'PC':<10} {'F1':<10} {'Status':<10}")
        print("-" * 70)
    
    threshold_results = {}
    best_threshold = None
    best_f1 = -1
    best_in_range = None
    best_f1_in_range = -1
    
    for threshold in distance_thresholds:
        # Apply clustering with this threshold
        cluster_labels, _ = apply_agglomerative_clustering(
            filtered_pairs_train,
            all_product_ids_train,
            dict_train,
            linkage=linkage,
            distance_threshold=threshold
        )
        
        # Evaluate clustering
        f1, precision, recall, TP, total_predicted_pairs, total_true_pairs = evaluate_clustering_f1_score(
            cluster_labels, dict_train, all_product_ids_train
        )
        
        n_clusters = len(set(cluster_labels))
        f1_max = f1_range[1] if isinstance(f1_range, tuple) else f1_range
        in_range = f1 <= f1_max
        
        threshold_results[threshold] = {
            'f1': f1,
            'precision': precision,
            'recall': recall,
            'TP': TP,
            'total_predicted_pairs': total_predicted_pairs,
            'total_true_pairs': total_true_pairs,
            'n_clusters': n_clusters,
            'in_range': in_range
        }
        
        status = "✓ IN RANGE" if in_range else "✗ OUT"
        
        if verbose:
            print(f"{threshold:<12.2f} {n_clusters:<10} {TP:<6} {precision:<10.6f} {recall:<10.6f} {f1:<10.6f} {status:<10}")
        
        # Track best overall (only if F1 <= f1_max)
        f1_max = f1_range[1] if isinstance(f1_range, tuple) else f1_range
        if f1 <= f1_max and f1 > best_f1:
            best_f1 = f1
            best_threshold = threshold
        
        # Track best within range (preferred to avoid overfitting)
        if in_range and f1 > best_f1_in_range:
            best_f1_in_range = f1
            best_in_range = threshold
    
    # Select best threshold (prefer in-range to avoid overfitting)
    f1_max = f1_range[1] if isinstance(f1_range, tuple) else f1_range
    if best_in_range is not None:
        selected_threshold = best_in_range
        selected_f1 = best_f1_in_range
        selection_reason = f"F1 <= {f1_max:.2f} ✓"
    else:
        selected_threshold = best_threshold
        selected_f1 = best_f1
        selection_reason = f"F1 > {f1_max:.2f} ✗ (no threshold found with F1 <= {f1_max:.2f})"
    
    if verbose:
        print("-" * 70)
        print(f"\nBest threshold: {selected_threshold} (F1 = {selected_f1:.6f}) [{selection_reason}]")
        f1_max = f1_range[1] if isinstance(f1_range, tuple) else f1_range
        if best_in_range is None:
            print(f"WARNING: No threshold found with F1 <= {f1_max:.2f}")
            print(f"Using best available: {best_threshold} (F1 = {best_f1:.6f})")
        print("="*70)
    
    return selected_threshold, selected_f1, threshold_results



In [11]:
SHOP_LIST = [
    'newegg', 'amazon', 'bestbuy', 'walmart', 'target', 'costco', 
    'bhphotovideo', 'tigerdirect', 'overstock', 'sears', 'ebay', 'thenerds'
]

BRAND_LIST = [
    "samsung", "lg", "sony", "philips", "panasonic", "sharp", "toshiba",
    "vizio", "hisense", "tcl", "insignia", "sanyo", "magnavox", "westinghouse",
    "element", "rca", "haier", "funai", "sceptre", "proscan", "coby",
    "supersonic", "naxa", "dynex", "emerson", "sylvania", "jvc", "hitachi"
]

def extract_brand_from_item(item, brand_list=BRAND_LIST):
    """Extract brand from title or featuresmap"""
    # Check title first
    title = item.get('title', '').lower()
    for b in brand_list:
        if b in title:
            return b
    
    # Check featuresmap
    features = item.get('featuresmap') or item.get('featuresMap')
    if isinstance(features, dict):
        brand_val = str(features.get('brand', '')).lower()
        for b in brand_list:
            if b in brand_val:
                return b
    return None


def extract_shop_from_item(item, shop_list=SHOP_LIST):
    """Extract shop/website from URL, title, or shop field"""
    # Check shop field first (most reliable)
    shop_field = item.get('shop', '') or item.get('Shop', '')
    if shop_field and isinstance(shop_field, str):
        shop_lower = shop_field.lower()
        for shop in shop_list:
            if shop in shop_lower:
                return shop
    
    # Check URL field
    url = item.get('url') or item.get('URL')
    if url and isinstance(url, str):
        url_lower = url.lower()
        for shop in shop_list:
            if shop in url_lower:
                return shop
    
    # Check title for shop names (may be removed in cleaned version)
    title = item.get('title', '') or item.get('Title', '')
    if title and isinstance(title, str):
        title_lower = title.lower()
        for shop in shop_list:
            if shop in title_lower:
                return shop
    return None

def filter_candidate_pairs_by_brand_and_shop(candidate_pairs, dict_data, 
                                              brand_list=BRAND_LIST, 
                                              shop_list=SHOP_LIST,
                                              dict_data_with_shops=None,
                                              verbose=True):
    
    filtered_pairs = set()
    removed_diff_brand = 0
    removed_same_shop = 0
    
    for i, j in candidate_pairs:
        pair_i, pair_j = i, j  # Alias for debug clarity
        item_i = dict_data[i]
        item_j = dict_data[j]
        
        # Extract brands
        brand_i = extract_brand_from_item(item_i, brand_list)
        brand_j = extract_brand_from_item(item_j, brand_list)
        
        # Rule 1: diffBrand() - Remove if both brands identified AND different
        if brand_i is not None and brand_j is not None and brand_i != brand_j:
            removed_diff_brand += 1
            continue
        
        # Extract shops
        # If dict_data has shops removed, try to get from dict_data_with_shops if provided
        if dict_data_with_shops is not None:
            shop_item_i = dict_data_with_shops.get(i, item_i)
            shop_item_j = dict_data_with_shops.get(j, item_j)
        else:
            shop_item_i = item_i
            shop_item_j = item_j
        
        shop_i = extract_shop_from_item(shop_item_i, shop_list)
        shop_j = extract_shop_from_item(shop_item_j, shop_list)
        
         # If dict_data has shops removed, try to get from dict_data_with_shops if provided
        if dict_data_with_shops is not None:
            shop_item_i = dict_data_with_shops.get(i, item_i)
            shop_item_j = dict_data_with_shops.get(j, item_j)
        else:
            shop_item_i = item_i
            shop_item_j = item_j
        
        shop_i = extract_shop_from_item(shop_item_i, shop_list)
        shop_j = extract_shop_from_item(shop_item_j, shop_list)
        
    
        # Rule 2: sameShop() - Remove if same shop (duplicates are cross-shop)
        if shop_i is not None and shop_j is not None and shop_i == shop_j:
            removed_same_shop += 1
            continue
        
        filtered_pairs.add((i, j))
    
    stats = {
        'original': len(candidate_pairs),
        'removed_diff_brand': removed_diff_brand,
        'removed_same_shop': removed_same_shop,
        'filtered': len(filtered_pairs)
    }
    
    if verbose:
        print(f"Original candidate pairs: {stats['original']}")
        print(f"Removed (different brands): {stats['removed_diff_brand']}")
        print(f"Removed (same shop): {stats['removed_same_shop']}")
        print(f"Filtered pairs remaining: {stats['filtered']}")
    
    return filtered_pairs, stats



In [12]:
# ============================================================================
# BOOTSTRAP EVALUATION: 5 Iterations with F1 Score Calculation
# ============================================================================
# This code structure runs bootstrap sampling 5 times and calculates F1 on test set
# Placeholders indicate where LSH, binary vectors, MinHashing should be inserted

import numpy as np
from collections import defaultdict

# Clean the data first
# Keep a version with shops for filtering later
all_values_dict_clean_keep_shops = clean_all_values_dict_advanced(all_values_dict, remove_shops=False)
all_values_dict_clean = clean_all_values_dict_advanced(all_values_dict, remove_shops=True)
n_samples = len(all_values_dict_clean)

print("="*70)
print("BOOTSTRAP EVALUATION: 5 Iterations")
print("="*70)
print(f"Total samples: {n_samples}")
print(f"Number of bootstrap iterations: 5")
print("="*70)

# Store results for each bootstrap iteration
bootstrap_results = []

# Run 5 bootstrap iterations
for bootstrap_idx in range(5):
    print(f"\n{'='*70}")
    print(f"BOOTSTRAP ITERATION {bootstrap_idx + 1}/5")
    print(f"{'='*70}")
    
    # ========================================================================
    # STEP 1: Create Bootstrap Train/Test Split
    # ========================================================================
    train_indices, test_indices = bootstrap_train_test_split(n_samples)
    
    # Get unique indices (remove duplicates from bootstrap sampling)
    unique_train_indices = np.unique(train_indices)
    unique_test_indices = np.unique(test_indices)
    
    # Create train and test dictionaries using only unique products
    dict_train = {i: all_values_dict_clean[idx] for i, idx in enumerate(unique_train_indices)}
    dict_test = {i: all_values_dict_clean[idx] for i, idx in enumerate(unique_test_indices)}
    
    print(f"\n--- Bootstrap {bootstrap_idx + 1} Split ---")
    print(f"  Training set: {len(train_indices)} samples ({len(unique_train_indices)} unique) -> using {len(dict_train)} unique products")
    print(f"  Test set: {len(test_indices)} samples ({len(unique_test_indices)} unique) -> using {len(dict_test)} unique products")
    
    # ========================================================================
    
    # ========================================================================
    # COMPARISON: Run both model word extraction methods
    # ========================================================================
    methods = [
        ('current', extract_all_model_words_paper_method),
        ('title_only', extract_model_words_title_only)
    ]
    
    for method_name, extract_func in methods:
        print(f"\n{'='*70}")
        print(f"METHOD: {method_name.upper()}")
        print(f"{'='*70}")
        
        # ========================================================================
        # STEP 2: Extract Model Words
        # ========================================================================
        # Extract model words from train and test sets separately
        model_words_train = []
        for idx, item in dict_train.items():
            mw = extract_func(item)
            model_words_train.append(mw)
        
        model_words_test = []
        for idx, item in dict_test.items():
            mw = extract_func(item)
            model_words_test.append(mw)
        
        print(f"\n--- Model Word Extraction ({method_name}) ---")
        print(f"  Extracted model words for {len(model_words_train)} training products")
        print(f"  Extracted model words for {len(model_words_test)} test products")
        
        # ========================================================================
        # Extract model words from train and test sets separately
        model_words_train = []
        for idx, item in dict_train.items():
           mw = extract_all_model_words_paper_method(item)
           model_words_train.append(mw)
    
        model_words_test = []
        for idx, item in dict_test.items():
            mw = extract_all_model_words_paper_method(item)
            model_words_test.append(mw)
    
        print("\n--- Model Word Extraction ---")
        print(f"  Extracted model words for {len(model_words_train)} training products")
        print(f"  Extracted model words for {len(model_words_test)} test products")
    
        # ========================================================================
        # STEP 3: Build Vocabulary
        # ========================================================================
        # Build vocabulary from BOTH training and test sets
        model_words_set_train = set()
        for mw_list in model_words_train:
            model_words_set_train.update(mw_list)
    
        model_words_set_test = set()
        for mw_list in model_words_test:
             model_words_set_test.update(mw_list)
    
        # Combined vocabulary
        combined_vocab = sorted(model_words_set_train | model_words_set_test)
        word_to_index = {word: idx for idx, word in enumerate(combined_vocab)}
        vocab = combined_vocab
    
        print("\n--- Vocabulary Building ---")
        print(f"  Training set unique model words: {len(model_words_set_train)}")
        print(f"  Test set unique model words: {len(model_words_set_test)}")
        print(f"  Combined vocabulary size: {len(vocab)}")
    
        # ========================================================================
        # STEP 3.5: Find Best Threshold on Training Set
        # ========================================================================
        best_threshold, best_f1_train, threshold_results = find_best_threshold_on_training_set(
        dict_train, model_words_train, vocab, word_to_index,
        thresholds=[0.3,0.2,0.4], f1_range=(0.0, 0.2), verbose=True
        )
    
        # Extract optimal (b, r) for the best threshold
        best_b = threshold_results[best_threshold]['best_b']
        best_r = threshold_results[best_threshold]['best_r']
    
        print(f"\n--- Best Threshold Selection ---")
        print(f"  Selected threshold: {best_threshold} (F1 on training: {best_f1_train:.6f})")
        print(f"  Optimal LSH parameters: b={best_b}, r={best_r}")
    
        # ========================================================================
        # STEP 4: Create Binary Vectors for Test Set
        # ========================================================================
        A_test = create_binary_vectors(model_words_test, vocab, word_to_index)
    
        print("\n--- Binary Vector Creation ---")
        print(f"  Test set binary vectors shape: {A_test.shape}")
    
        # ========================================================================
        # STEP 5: Compute MinHash Signatures for Test Set
        # ========================================================================
        L_test, _ = compute_minhash_signatures(A_test)
    
        print("\n--- MinHash Signature Computation ---")
        print(f"  Test set signature matrix shape: {L_test.shape}")
    
        # ========================================================================
        # STEP 6: Build Ground Truth for Test Set (needed for all thresholds)
        # ========================================================================
        modelID_to_indices_test = defaultdict(list)
        for idx, item in dict_test.items():
            if 'modelID' in item:
                modelID_to_indices_test[item['modelID']].append(idx)
    
        # Keep only modelIDs with 2+ products (actual duplicates)
        duplicate_clusters_test = {mid: set(idxs) for mid, idxs in modelID_to_indices_test.items() if len(idxs) > 1}
    
        # Calculate total true duplicate pairs in test set
        total_true_pairs_test = sum(len(idxs) * (len(idxs) - 1) // 2 for idxs in duplicate_clusters_test.values())
    
        print(f"\n--- Ground Truth (Test Set) ---")
        print(f"  Duplicate clusters: {len(duplicate_clusters_test)}")
        print(f"  Total true duplicate pairs: {total_true_pairs_test}")
    
        # ========================================================================
        # STEP 7: Test Multiple Thresholds on Test Set for Smooth Curve
        # ========================================================================
        # Test multiple thresholds to get data points across different fractions
        test_thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
        product_ids_test = list(dict_test.keys())
    
        print(f"\n--- Testing Multiple Thresholds on Test Set ---")
        print(f"  Testing {len(test_thresholds)} thresholds: {test_thresholds}")
    
        # Store results for each threshold
        for threshold in test_thresholds:
        # Apply LSH banding with this threshold
             candidate_pairs = lsh_banding(L_test, product_ids_test, t=threshold, verbose=False)
        
        # Count true duplicates
        TP = count_true_duplicates_in_candidates(dict_test, candidate_pairs)
        
        # Calculate metrics
        PQ = TP / len(candidate_pairs) if len(candidate_pairs) > 0 else 0.0
        PC = TP / total_true_pairs_test if total_true_pairs_test > 0 else 0.0
        F1 = 2 * (PQ * PC) / (PQ + PC) if (PQ + PC) > 0 else 0.0
        
        # Store results for this threshold
        bootstrap_results.append({
        'iteration': bootstrap_idx + 1,
        'threshold': threshold,
        'train_size': len(train_indices),
        'train_unique': len(np.unique(train_indices)),
        'test_size': len(test_indices),
        'candidate_pairs': len(candidate_pairs),
        'TP': TP,
        'PQ': PQ,
        'PC': PC,
        'F1': F1,
        'total_true_pairs': total_true_pairs_test,
            'method': method_name})
    
        print(f"  Completed testing {len(test_thresholds)} thresholds")
    
        # ========================================================================
        # STEP 8: Also evaluate with best threshold from training (for comparison)
        # ========================================================================
        # Use theoretical (b, r) values calculated from best threshold: t = (1/b)^(1/r)
        # These are the same (b, r) used on training set for this threshold
        candidate_pairs_best = lsh_banding(L_test, product_ids_test, t=best_threshold, b=best_b, r=best_r, verbose=False)
        TP_best = count_true_duplicates_in_candidates(dict_test, candidate_pairs_best)
        PQ_best = TP_best / len(candidate_pairs_best) if len(candidate_pairs_best) > 0 else 0.0
        PC_best = TP_best / total_true_pairs_test if total_true_pairs_test > 0 else 0.0
        F1_best = 2 * (PQ_best * PC_best) / (PQ_best + PC_best) if (PQ_best + PC_best) > 0 else 0.0
    
        print(f"\n--- Evaluation with Best Threshold (for reference) ---")
        print(f"  Threshold used: {best_threshold} (selected from training set)")
        print(f"  LSH parameters: b={best_b}, r={best_r} (optimized on training set)")
        print(f"  Candidate pairs: {len(candidate_pairs_best)}")
        print(f"  True positives (TP): {TP_best}")
        print(f"  Pair Quality (PQ): {PQ_best:.6f} ({PQ_best*100:.4f}%)")
        print(f"  Pair Completeness (PC): {PC_best:.6f} ({PC_best*100:.4f}%)")
        print(f"  F1 Score: {F1_best:.6f} ({F1_best*100:.4f}%)")
    
        # Use best threshold candidate pairs for filtering (if needed)
        candidate_pairs_for_eval = candidate_pairs_best
    
        # ========================================================================
        # STEP 10: Apply Filtering to Test Set
        # ========================================================================
        # Create dict_test_with_shops for shop extraction (shops preserved)
        # Map from dict_test keys (0, 1, 2, ...) to original indices in all_values_dict_clean_keep_shops
        dict_test_with_shops = {i: all_values_dict_clean_keep_shops[test_indices[i]] for i in dict_test.keys()}
    
        print(f"\n--- Filtering Test Set Candidate Pairs ---")
        filtered_pairs_test, filter_stats_test = filter_candidate_pairs_by_brand_and_shop(
        candidate_pairs_for_eval,
        dict_test,
        dict_data_with_shops=dict_test_with_shops,
        verbose=True
        )
    
        # ========================================================================
        # STEP 10.5: Clustering will be applied AFTER optimization (see STEP 11.6)
        # ========================================================================
        # (Moved to after training set optimization to use optimized threshold)
        # STEP 11: Apply Filtering to Training Set
        # ========================================================================
        # First, get candidate pairs from training set using best threshold
        A_train = create_binary_vectors(model_words_train, vocab, word_to_index)
        L_train, _ = compute_minhash_signatures(A_train)
        product_ids_train = list(dict_train.keys())
        candidate_pairs_train = lsh_banding(L_train, product_ids_train, t=best_threshold, verbose=False)
    
        # Create dict_train_with_shops for shop extraction (shops preserved)
        # Map from dict_train keys (0, 1, 2, ...) to original indices in all_values_dict_clean_keep_shops
        dict_train_with_shops = {i: all_values_dict_clean_keep_shops[train_indices[i]] for i in dict_train.keys()}
    
        print(f"\n--- Filtering Training Set Candidate Pairs ---")
        filtered_pairs_train, filter_stats_train = filter_candidate_pairs_by_brand_and_shop(
        candidate_pairs_train,
        dict_train,
        dict_data_with_shops=dict_train_with_shops,
        verbose=True
        )
    
        # ========================================================================
        # STEP 11.5: Optimize Clustering Distance Threshold on Training Set
        # ========================================================================
        print(f"\n--- Optimizing Clustering Distance Threshold on Training Set ---")
    
        # Get all product IDs from training set
        all_product_ids_train = list(dict_train.keys())
    
        # Find best distance threshold on training set
        best_clustering_threshold, best_clustering_f1_train, clustering_threshold_results = find_best_clustering_distance_threshold(
        filtered_pairs_train,
        all_product_ids_train,
        dict_train,
        linkage='complete',
        distance_thresholds=[0.2, 0.3, 0.4, 0.5, 0.6, 0.7],
        f1_range=(0.0, 0.2),  # F1 upper bound constraint (F1 <= 0.2) to prevent overfitting
        verbose=True
        )
    
        print(f"\n--- Best Clustering Distance Threshold Selected ---")
        print(f"  Distance threshold: {best_clustering_threshold}")
        print(f"  F1 Score on Training: {best_clustering_f1_train:.6f}")
        print(f"  (This means similarity >= {1.0 - best_clustering_threshold:.2f} for clustering)")
    
        # ========================================================================
        # STEP 11.6: Apply Agglomerative Clustering to Test Set (with Optimized Threshold)
        # ========================================================================
        print(f"\n--- Agglomerative Clustering on Test Set (with Optimized Threshold) ---")
    
        # Get all product IDs from test set
        all_product_ids_test = list(dict_test.keys())
    
        # Apply clustering with optimized distance threshold from training set
        # NOTE: Uses TMWM similarity computed as overlap coefficient: |model_words_1 ∩ model_words_2| / min(|model_words_1|, |model_words_2|)
        print(f"  Using optimized distance threshold: {best_clustering_threshold} (F1 on training: {best_clustering_f1_train:.6f})")
        print(f"  This means similarity >= {1.0 - best_clustering_threshold:.2f} for clustering")
    
        cluster_labels, clustering_model = apply_agglomerative_clustering(
        filtered_pairs_test,
        all_product_ids_test,
        dict_test,  # Pass dict_data for TMWM similarity computation
        linkage='complete',  # Complete linkage = maximum linkage (double linkage)
        distance_threshold=best_clustering_threshold  # Use optimized threshold from training set
        )
    
        # Evaluate clustering using F1 score
        f1_clustering, precision_clustering, recall_clustering, TP_clustering, \
        total_predicted_pairs, total_true_pairs_clustering = evaluate_clustering_f1_score(
        cluster_labels,
        dict_test,
        all_product_ids_test
        )
    
        print(f"\n  Clustering Results (Test Set):")
        print(f"    Number of clusters: {len(set(cluster_labels))}")
        print(f"    Clusters with 2+ products: {sum(1 for label in set(cluster_labels) if list(cluster_labels).count(label) > 1)}")
        print(f"    True positives (TP): {TP_clustering}")
        print(f"    Total predicted pairs: {total_predicted_pairs}")
        print(f"    Total true pairs: {total_true_pairs_clustering}")
        print(f"    Precision (PQ): {precision_clustering:.6f} ({precision_clustering*100:.4f}%)")
        print(f"    Recall (PC): {recall_clustering:.6f} ({recall_clustering*100:.4f}%)")
        print(f"    F1 Score: {f1_clustering:.6f} ({f1_clustering*100:.4f}%)")
    
        # Check for overfitting: if test F1 is much lower than training F1, warn
        f1_gap = best_clustering_f1_train - f1_clustering
        if f1_gap > 0.10:  # More than 10% gap suggests overfitting
            print(f"\n  WARNING: Potential overfitting detected!")
            print(f"    Training F1: {best_clustering_f1_train:.6f}")
            print(f"    Test F1: {f1_clustering:.6f}")
            print(f"    Gap: {f1_gap:.6f} ({f1_gap*100:.2f}%)")
        else:
            print(f"\n  Overfitting check: Gap = {f1_gap:.6f} ({f1_gap*100:.2f}%) - OK")
    
        # Store clustering results for the best threshold evaluation
        # Find the result entry that matches the best threshold and current iteration
        if len(bootstrap_results) > 0:
            current_iteration = bootstrap_idx + 1
        for result in reversed(bootstrap_results):
            if result.get('threshold') == best_threshold and result.get('iteration') == current_iteration:
                result['clustering_f1'] = f1_clustering
                result['clustering_precision'] = precision_clustering
                result['clustering_recall'] = recall_clustering
                result['clustering_TP'] = TP_clustering
                result['clustering_n_clusters'] = len(set(cluster_labels))
                result['clustering_total_predicted_pairs'] = total_predicted_pairs
                result['clustering_total_true_pairs'] = total_true_pairs_clustering
                result['clustering_f1_train'] = best_clustering_f1_train
                result['clustering_f1_gap'] = f1_gap
        break
    
        # Store filtered results
        bootstrap_results[-1]['filtered_pairs_test'] = len(filtered_pairs_test)
        bootstrap_results[-1]['filtered_pairs_train'] = len(filtered_pairs_train)
        bootstrap_results[-1]['filter_stats_test'] = filter_stats_test
        bootstrap_results[-1]['filter_stats_train'] = filter_stats_train
        bootstrap_results[-1]['best_clustering_threshold'] = best_clustering_threshold
        bootstrap_results[-1]['best_clustering_f1_train'] = best_clustering_f1_train

        # ========================================================================
        
# STEP 12: Aggregate Results Across All Bootstrap Iterations
# ========================================================================
print("\n" + "="*70)
        
    
print("BOOTSTRAP SUMMARY: Aggregated Results Across 5 Iterations")
print("="*70)

F1_scores = [r['F1'] for r in bootstrap_results]
PQ_scores = [r['PQ'] for r in bootstrap_results]
PC_scores = [r['PC'] for r in bootstrap_results]

print(f"\nF1 Score Statistics:")
print(f"  Mean: {np.mean(F1_scores):.6f} ({np.mean(F1_scores)*100:.4f}%)")
print(f"  Std:  {np.std(F1_scores):.6f} ({np.std(F1_scores)*100:.4f}%)")
print(f"  Min:  {np.min(F1_scores):.6f} ({np.min(F1_scores)*100:.4f}%)")
print(f"  Max:  {np.max(F1_scores):.6f} ({np.max(F1_scores)*100:.4f}%)")

print(f"\nPair Quality (PQ) Statistics:")
print(f"  Mean: {np.mean(PQ_scores):.6f} ({np.mean(PQ_scores)*100:.4f}%)")
print(f"  Std:  {np.std(PQ_scores):.6f} ({np.std(PQ_scores)*100:.4f}%)")

print(f"\nPair Completeness (PC) Statistics:")
print(f"  Mean: {np.mean(PC_scores):.6f} ({np.mean(PC_scores)*100:.4f}%)")
print(f"  Std:  {np.std(PC_scores):.6f} ({np.std(PC_scores)*100:.4f}%)")

print(f"\nDetailed Results per Iteration:")
print(f"{'Iter':<6} {'Threshold':<10} {'F1':<12} {'PQ':<12} {'PC':<12} {'TP':<6} {'Candidates':<12}")
print("-" * 80)
for r in bootstrap_results:
    print(f"{r['iteration']:<6} {r.get('threshold', 'N/A'):<10.2f} {r['F1']:<12.6f} {r['PQ']:<12.6f} {r['PC']:<12.6f} "
          f"{r['TP']:<6} {r['candidate_pairs']:<12}")

print("="*70)

# Store final aggregated results
bootstrap_summary = {
    'n_iterations': 5,
    'F1_mean': np.mean(F1_scores),
    'F1_std': np.std(F1_scores),
    'F1_min': np.min(F1_scores),
    'F1_max': np.max(F1_scores),
    'PQ_mean': np.mean(PQ_scores),
    'PQ_std': np.std(PQ_scores),
    'PC_mean': np.mean(PC_scores),
    'PC_std': np.std(PC_scores),
    'all_results': bootstrap_results
}

print(f"\nBootstrap evaluation complete!")
print(f"Final F1 Score: {bootstrap_summary['F1_mean']:.6f} ± {bootstrap_summary['F1_std']:.6f}")
print("="*70)



BOOTSTRAP EVALUATION: 5 Iterations
Total samples: 1624
Number of bootstrap iterations: 5

BOOTSTRAP ITERATION 1/5

--- Bootstrap 1 Split ---
  Training set: 1624 samples (1048 unique) -> using 1048 unique products
  Test set: 576 samples (576 unique) -> using 576 unique products

METHOD: CURRENT

--- Model Word Extraction (current) ---
  Extracted model words for 1048 training products
  Extracted model words for 576 test products

--- Model Word Extraction ---
  Extracted model words for 1048 training products
  Extracted model words for 576 test products

--- Vocabulary Building ---
  Training set unique model words: 1680
  Test set unique model words: 1113
  Combined vocabulary size: 2186

FINDING BEST THRESHOLD ON TRAINING SET
F1 score upper bound constraint: F1 <= 0.20
Testing thresholds: [0.3, 0.2, 0.4]
Training set size: 1048 products
Total true duplicate pairs in training set: 169
Duplicate clusters: 153

Testing thresholds...
Threshold    Candidates   TP     PQ         PC     

In [24]:
# ============================================================================
# CLUSTERING WITHOUT LSH: Bootstrap Evaluation (5 Iterations)
# ============================================================================
# This evaluates clustering directly on all products without using LSH
# to generate candidate pairs first.

import numpy as np
from collections import defaultdict
from sklearn.cluster import AgglomerativeClustering

def build_full_similarity_matrix(all_product_ids, dict_data, verbose=True):
    """
    Build a full similarity matrix for ALL products (not just filtered pairs).
    
    Parameters:
    -----------
    all_product_ids : list
        List of all product IDs
    dict_data : dict
        Dictionary mapping product IDs to product data
    verbose : bool
        Whether to print progress
    
    Returns:
    --------
    similarity_matrix : np.ndarray
        Full TMWM similarity matrix for all products
    product_id_to_index : dict
        Mapping from product ID to matrix index
    """
    n = len(all_product_ids)
    product_id_to_index = {pid: idx for idx, pid in enumerate(all_product_ids)}
    
    # Initialize similarity matrix (all zeros)
    similarity_matrix = np.zeros((n, n), dtype=float)
    
    # Set diagonal to 1.0 (self-similarity)
    np.fill_diagonal(similarity_matrix, 1.0)
    
    # Compute TMWM similarity for ALL pairs
    total_pairs = n * (n - 1) // 2
    computed = 0
    
    if verbose:
        print(f"  Computing similarity for {total_pairs} pairs...")
    
    for i in range(n):
        for j in range(i + 1, n):
            product_i = all_product_ids[i]
            product_j = all_product_ids[j]
            
            item_i = dict_data.get(product_i)
            item_j = dict_data.get(product_j)
            
            if item_i and item_j:
                # Compute TMWM similarity
                tmwm_sim = compute_tmwm_similarity(item_i, item_j)
                similarity_matrix[i, j] = tmwm_sim
                similarity_matrix[j, i] = tmwm_sim  # Symmetric
                computed += 1
        
        if verbose and (i + 1) % 100 == 0:
            print(f"    Computed similarities for {i + 1}/{n} products...")
    
    if verbose:
        print(f"  Completed computing TMWM similarity for {computed} pairs")
    
    return similarity_matrix, product_id_to_index


# Clean the data first
all_values_dict_clean = clean_all_values_dict_advanced(all_values_dict, remove_shops=True)
n_samples = len(all_values_dict_clean)

print("="*70)
print("CLUSTERING WITHOUT LSH: Bootstrap Evaluation (5 Iterations)")
print("="*70)
print(f"Total samples: {n_samples}")
print(f"Number of bootstrap iterations: 5")
print("="*70)

# Store results for each bootstrap iteration
clustering_no_lsh_results = []

# Run 5 bootstrap iterations
for bootstrap_idx in range(5):
    print(f"\n{'='*70}")
    print(f"BOOTSTRAP ITERATION {bootstrap_idx + 1}/5")
    print(f"{'='*70}")
    
    # ========================================================================
    # STEP 1: Create Bootstrap Train/Test Split
    # ========================================================================
    train_indices, test_indices = bootstrap_train_test_split(n_samples)
    
    # Get unique indices (remove duplicates from bootstrap sampling)
    unique_train_indices = np.unique(train_indices)
    unique_test_indices = np.unique(test_indices)
    
    # Create train and test dictionaries using only unique products
    dict_train = {i: all_values_dict_clean[idx] for i, idx in enumerate(unique_train_indices)}
    dict_test = {i: all_values_dict_clean[idx] for i, idx in enumerate(unique_test_indices)}
    
    print(f"\n--- Bootstrap {bootstrap_idx + 1} Split ---")
    print(f"  Training set: {len(train_indices)} samples ({len(unique_train_indices)} unique)")
    print(f"  Test set: {len(test_indices)} samples ({len(unique_test_indices)} unique)")
    
    # ========================================================================
    # STEP 2: Find Best Clustering Threshold on Training Set
    # ========================================================================
    print(f"\n--- Finding Best Clustering Threshold on Training Set ---")
    
    all_product_ids_train = list(dict_train.keys())
    
    # Build full similarity matrix for training set
    print("  Building full similarity matrix for training set...")
    similarity_matrix_train, _ = build_full_similarity_matrix(
        all_product_ids_train, dict_train, verbose=True
    )
    
    # Convert similarity to distance
    distance_matrix_train = 1.0 - similarity_matrix_train
    
    # Test different distance thresholds on training set
    distance_thresholds = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
    best_threshold = None
    best_f1_train = -1
    threshold_results_train = {}
    
    print(f"\n  Testing {len(distance_thresholds)} distance thresholds on training set...")
    print(f"  {'Threshold':<12} {'Clusters':<10} {'TP':<6} {'PQ':<10} {'PC':<10} {'F1':<10}")
    print("  " + "-" * 68)
    
    for dist_thresh in distance_thresholds:
        # Apply clustering with this threshold
        clustering = AgglomerativeClustering(
            n_clusters=None,
            metric='precomputed',
            linkage='complete',
            distance_threshold=dist_thresh,
            compute_full_tree=True
        )
        
        cluster_labels = clustering.fit_predict(distance_matrix_train)
        
        # Evaluate clustering
        f1, precision, recall, TP, total_predicted_pairs, total_true_pairs = evaluate_clustering_f1_score(
            cluster_labels, dict_train, all_product_ids_train
        )
        
        threshold_results_train[dist_thresh] = {
            'f1': f1,
            'precision': precision,
            'recall': recall,
            'TP': TP,
            'n_clusters': len(set(cluster_labels)),
            'predicted_pairs': total_predicted_pairs,
            'true_pairs': total_true_pairs
        }
        
        print(f"  {dist_thresh:<12.2f} {len(set(cluster_labels)):<10} {TP:<6} {precision:<10.6f} {recall:<10.6f} {f1:<10.6f}")
        
        # Update best threshold (prefer F1 <= 0.2 to avoid overfitting)
        if f1 <= 0.2 and f1 > best_f1_train:
            best_f1_train = f1
            best_threshold = dist_thresh
    
    if best_threshold is None:
        # If no threshold found with F1 <= 0.2, use the best available
        best_threshold = max(threshold_results_train.keys(), key=lambda t: threshold_results_train[t]['f1'])
        best_f1_train = threshold_results_train[best_threshold]['f1']
    
    print(f"\n  Best threshold: {best_threshold} (F1 on training: {best_f1_train:.6f})")
    
    # ========================================================================
    # STEP 3: Apply Clustering to Test Set with Best Threshold
    # ========================================================================
    print(f"\n--- Applying Clustering to Test Set ---")
    
    all_product_ids_test = list(dict_test.keys())
    
    # Build full similarity matrix for test set
    print("  Building full similarity matrix for test set...")
    similarity_matrix_test, _ = build_full_similarity_matrix(
        all_product_ids_test, dict_test, verbose=True
    )
    
    # Convert similarity to distance
    distance_matrix_test = 1.0 - similarity_matrix_test
    
    # Apply clustering with best threshold
    print(f"  Applying clustering with distance threshold: {best_threshold}")
    clustering = AgglomerativeClustering(
        n_clusters=None,
        metric='precomputed',
        linkage='complete',
        distance_threshold=best_threshold,
        compute_full_tree=True
    )
    
    cluster_labels = clustering.fit_predict(distance_matrix_test)
    
    # Evaluate clustering on test set
    f1_test, precision_test, recall_test, TP_test, total_predicted_pairs_test, total_true_pairs_test = evaluate_clustering_f1_score(
        cluster_labels, dict_test, all_product_ids_test
    )
    
    print(f"\n  Clustering Results (Test Set):")
    print(f"    Number of clusters: {len(set(cluster_labels))}")
    print(f"    Clusters with 2+ products: {sum(1 for c in set(cluster_labels) if list(cluster_labels).count(c) > 1)}")
    print(f"    True positives (TP): {TP_test}")
    print(f"    Total predicted pairs: {total_predicted_pairs_test}")
    print(f"    Total true pairs: {total_true_pairs_test}")
    print(f"    Precision (PQ): {precision_test:.6f} ({precision_test*100:.4f}%)")
    print(f"    Recall (PC): {recall_test:.6f} ({recall_test*100:.4f}%)")
    print(f"    F1 Score: {f1_test:.6f} ({f1_test*100:.4f}%)")
    
    # Store results
    clustering_no_lsh_results.append({
        'iteration': bootstrap_idx + 1,
        'best_threshold': best_threshold,
        'best_f1_train': best_f1_train,
        'f1_test': f1_test,
        'precision_test': precision_test,
        'recall_test': recall_test,
        'TP_test': TP_test,
        'total_predicted_pairs_test': total_predicted_pairs_test,
        'total_true_pairs_test': total_true_pairs_test,
        'n_clusters': len(set(cluster_labels)),
        'train_size': len(unique_train_indices),
        'test_size': len(unique_test_indices)
    })
    
    print(f"\n--- Bootstrap Iteration {bootstrap_idx + 1} Complete ---")

# ========================================================================
# STEP 4: Summary Statistics
# ========================================================================
print("\n" + "="*70)
print("CLUSTERING WITHOUT LSH: SUMMARY STATISTICS")
print("="*70)

if clustering_no_lsh_results:
    f1_scores = [r['f1_test'] for r in clustering_no_lsh_results]
    precision_scores = [r['precision_test'] for r in clustering_no_lsh_results]
    recall_scores = [r['recall_test'] for r in clustering_no_lsh_results]
    
    print(f"\nF1 Score Statistics:")
    print(f"  Mean: {np.mean(f1_scores):.6f} ± {np.std(f1_scores):.6f} ({np.mean(f1_scores)*100:.4f}% ± {np.std(f1_scores)*100:.4f}%)")
    print(f"  Min:  {np.min(f1_scores):.6f} ({np.min(f1_scores)*100:.4f}%)")
    print(f"  Max:  {np.max(f1_scores):.6f} ({np.max(f1_scores)*100:.4f}%)")
    
    print(f"\nPrecision (PQ) Statistics:")
    print(f"  Mean: {np.mean(precision_scores):.6f} ± {np.std(precision_scores):.6f}")
    
    print(f"\nRecall (PC) Statistics:")
    print(f"  Mean: {np.mean(recall_scores):.6f} ± {np.std(recall_scores):.6f}")
    
    print(f"\nDetailed Results per Iteration:")
    print(f"{'Iter':<6} {'Thresh':<8} {'F1 Train':<10} {'F1 Test':<10} {'PQ':<10} {'PC':<10} {'TP':<6} {'Clusters':<10}")
    print("-" * 80)
    for r in clustering_no_lsh_results:
        print(f"{r['iteration']:<6} {r['best_threshold']:<8.2f} {r['best_f1_train']:<10.6f} {r['f1_test']:<10.6f} "
              f"{r['precision_test']:<10.6f} {r['recall_test']:<10.6f} {r['TP_test']:<6} {r['n_clusters']:<10}")
    
    print("="*70)
else:
    print("No results found.")


CLUSTERING WITHOUT LSH: Bootstrap Evaluation (5 Iterations)
Total samples: 1624
Number of bootstrap iterations: 5

BOOTSTRAP ITERATION 1/5

--- Bootstrap 1 Split ---
  Training set: 1624 samples (1016 unique)
  Test set: 608 samples (608 unique)

--- Finding Best Clustering Threshold on Training Set ---
  Building full similarity matrix for training set...
  Computing similarity for 515620 pairs...
    Computed similarities for 100/1016 products...
    Computed similarities for 200/1016 products...
    Computed similarities for 300/1016 products...
    Computed similarities for 400/1016 products...
    Computed similarities for 500/1016 products...
    Computed similarities for 600/1016 products...
    Computed similarities for 700/1016 products...
    Computed similarities for 800/1016 products...
    Computed similarities for 900/1016 products...
    Computed similarities for 1000/1016 products...
  Completed computing TMWM similarity for 515620 pairs

  Testing 7 distance thresholds